In this notebook we simulate hard rhombic dodecahedra using the [HPMC component](https://hoomd-blue.readthedocs.io/en/stable/package-hpmc.html) of HOOMD-Blue. We visualize the shapes using the [fresnel backend](https://fresnel.readthedocs.io/en/stable/) and color them by their [Steinhardt](https://freud.readthedocs.io/en/stable/order.html#freud.order.LocalQlNear) $Q_6$ order parameter.

In [ ]:
import flowws
import gtar
from hoomd_flowws.Init import Init
from hoomd_flowws.RunHPMC import RunHPMC
from hoomd_flowws.ShapeDefinition import ShapeDefinition

import json
import plato, plato.draw.fresnel as draw
import freud
import numpy as np
import IPython
import ipywidgets

In [ ]:
shape = dict(type='icosahedron', modifications=[dict(type='scale', factor=2)])

storage = flowws.DirectoryStorage()
stages = [
    ShapeDefinition(shape_arguments=[shape]),
    Init(number=128),
    RunHPMC(steps=1e5, integrator='npt', box_move_length=(.05, 1), dump_period=1e3),
]

flowws.Workflow(stages, storage, dict(hoomd_context_args='--mode=cpu')).run()

In [ ]:
num_frames = 0

def get_frame(frame=-1):
    global num_frames
    with gtar.GTAR('dump.sqlite', 'r') as traj:
        (posRec, quatRec, boxRec), frames = traj.framesWithRecordsNamed(['position', 'orientation', 'box'])

        for (_, type_shapes) in traj.recordsNamed('type_shapes.json'):
            type_shapes = json.loads(type_shapes)

        num_frames = len(frames)
        positions = traj.getRecord(posRec, frames[frame])
        orientations = traj.getRecord(quatRec, frames[frame])
        box = traj.getRecord(boxRec, frames[frame])

    shapedef = type_shapes[0]
    return positions, orientations, box, shapedef

In [ ]:
def update(scene, frame=-1):
    (positions, orientations, box, shapedef) = get_frame(frame)
    # get Steinhardt q6, rescaled from 0 to 1, as cval
    fbox = freud.box.Box.from_box(box)
    nn = freud.locality.NearestNeighbors(2, 12).compute(fbox, positions, positions)
    q6 = freud.order.LocalQl(fbox, 8, 12).compute(positions, nn.nlist)
    cval = q6.Ql.copy()
    cval -= np.min(cval)
    cval /= np.max(cval)

    colors = plato.cmap.cubehelix(.25 + .5*cval, s=2)
    
    for prim in scene:
        prim.positions = positions
        prim.orientations = orientations
        prim.colors = colors
        prim.vertices = shapedef['vertices']
        
prim = draw.ConvexPolyhedra(outline=.025)
features = dict(pathtracer=dict(samples=128))
scene = draw.Scene(prim, zoom=1.8, features=features,
                   rotation=[0.94183356, 0.15549502, 0.29752356, 0.01583402])
update(scene)

target = '../../gallery/flowws_icosahedra_fresnel.png'
scene.save(target)
IPython.display.Image(filename=target)

In [ ]:
import plato.draw.vispy as interactive

live_scene = scene.convert(interactive)
live_scene.show()

@ipywidgets.interact(frame=(0, num_frames - 1))
def plot(frame=0):
    update(live_scene, frame)
    live_scene.render()